## Module 1 - Lab

Data Proprocessing and exploration is a crucial step in data analysis. Often times, data is not present in a ready to use format. Its dirty and scattered in different places. Data analysis has to be done before applying any data science on data. Data can be in different formats like CSV, text, HTML, XML, JSON, ASCII, image etc. This notebook demonstrates how to process a json data file to convert it into a csv file.

Each record in a JSON file is called a document. Each document can have different number of fields, and some of the fields can be nested within other fields. This unstructured data is often stored in JavaScript Object Notation (JSON). JSON is a way to encode data structures like lists and dictionaries to strings that ensures that they are easily readable by machines.

Go to wiki to read more about [Json data format](https://en.wikipedia.org/wiki/JSON)

Python has great JSON support, with the json library. Python lists and dictionaries can be converted to JSON, and convert strings to lists and dictionaries. JSON data looks much like a dictionary would in Python, with keys and values stored.

### JSON example:  

[From wiki] Possible JSON representation describing a person.

    {
      "firstName": "John",
      "lastName": "Smith",
      "isAlive": true,
      "age": 25,
      "address": {
        "streetAddress": "21 2nd Street",
        "city": "New York",
        "state": "NY",
        "postalCode": "10021-3100"
      },
      "phoneNumbers": [
        {
          "type": "home",
          "number": "212 555-1234"
        },
        {
          "type": "office",
          "number": "646 555-4567"
        },
        {
          "type": "mobile",
          "number": "123 456-7890"
        }
      ],
      "children": [],
      "spouse": null
    }

This notebook leverages python library **`Pandas`** to explore and build csv file out of a text file with json documents.


Read the data from **`../../../datasets/chicago_small_loans/chicago_small_loans/loans.txt`**. Use json package to load the json format data. 

In [ ]:
import json
with open('../../../datasets/chicago_small_loans/loans.txt', 'r') as f:
    loans_data=json.load(f)

### The Dataset

The dataset used here contains information on Chicago Microlending Institute (CMI) Microloans. The data can be downloaded from [here](https://data.cityofchicago.org/Community-Economic-Development/Chicago-Microlending-Institute-CMI-Microloans/dpkg-upyz). The data contains information about the loans disbursed and reflects the lender, location, business industry, and borrower demographics. 

In [ ]:
type(loans_data)

Now **`loans`** is a json object and is stored as a dictionary with key, value pairs in python. **`loans`** is a nested json object with data and meta as its parent tags. We are interested in data tag. So we just have to extract contents of data tag. Have a look at the data inside **`loans`**. 

In [ ]:
# To see the main tags in loans json object
for tags in loans_data:
    print(tags)

In [ ]:
loans_data

This loans_data needs to be serialized(flattened) like an array so that it can be read as a list. The **`json_normalize`** method in pandas package does exactly the same. **`pandas.io.json.json_normalize`** method returns a pandas dataframe out of json format data file.


**Reference: ** 
- [json_normalize()](https://medium.com/@amirziai/flattening-json-objects-in-python-f5343c794b10#.22pg1vyfy)
- [pandas.io.json.json_normalize()](http://pandas.pydata.org/pandas-docs/version/0.19.2/generated/pandas.io.json.json_normalize.html)

### json_normalize()

**Parameters:	**
- data : dict or list of dicts unserialized JSON objects
- record_path : string or list of strings, default None. Path in each object to list of records. If not passed, data will be                     assumed to be an array of records
- meta : list of paths (string or list of strings), default None. Fields to use as metadata for each record in resulting table
- record_prefix : string, default None. If True, prefix records with dotted (?) path, e.g. foo.bar.field if path to records is                     [‘foo’, ‘bar’]
- meta_prefix : string, default None

**Returns: DataFrame**

Note: Also, the package **`flatten_json`** has functions to flatten a json object.

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

result=pd.io.json.json_normalize(loans_data,'data') # parameter data is passed because we only want the contents 
                                                    # of data tag
result

The array in the last column which has location information is not flattened yet. The **`meta`** parameter of json_normalize() function takes arguments, string or list of strings fields to use as metadata for each record in resulting table. meta will tell python how to parse the json(the structure to follow and create a flat array out of it). This allows for configuring the structure of the output file. This helps us in structuring the output file/dataframe but the problem is that the input object must be consistent and the arguments have to be picked manually depending on the structure. 

A json object doesn't have a definite structure. Thats the beauty and also paining aspect of Json. You cannot define the structure how you want to extract the json object data that fits every every document.  As shown [here](https://medium.com/@amirziai/flattening-json-objects-in-python-f5343c794b10#.22pg1vyfy) 

    sample_object = {'Name':'John', 'Location':{'City':'Los Angeles','State':'CA'}}

                                                and

    sample_object = {'Name':'John', 'Location':{'City':'Los Angeles','State':'CA'}, 'hobbies':['Music', 'Running']}

could be records in the same dataset. They both have different structure. Below defined python function will help in flattening JSON objects of arbitrary structure.

In [ ]:
# out{} is a empty dictionary. 
def flatten_json(input_doc):
    out = {}

# flatten() is called for every key value pair and checked for its type either dictionatry or list.  

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
#                 print(a,name)
                flatten(a, name + str(i) + '_')
                i += 1
        else:
#             print(name)
            out[name[:-1]] = x

    flatten(input_doc)
    return out

If you take the first json document

<img src="../images/json.PNG">

The function flatten_json() will recive this whole document as input in input_doc. It is checked whether its a list or dictionary primarily. Run below command to verify its a list. Now the **`elif`** condition for list is satisfied. Now the for loop is executed for each value in the list for 22 times since there are 22 elements in the list including the final array. The whole array at the end is the 22nd element. 

So the first document "loans_data["data"][0]" is checked if its a dictionary or a list. Since its a list, the for lopp is iterated for 22 times calling the same function recursively for 22 times. The first value "1" sent as input to flatten function again. since this time "1" is neither list nor a dictionary final else condition will be true. So **"1"** is appended to out{}. name has value ""+str(1)+"_" which will be a string "1_". But **"name[:-1]"** in final else statement strips final "_" if there are a series underscore values. Look at the output below to understand whats going on. 

Same applied to second value "CAB3E0EA-54FF-4502-902A-092B10ECDAF8" in next iteration. In last iteration the value is a list. verify the type by running below code cell. The flatten function is recursively called for 5 elements in inner list and name retainjs the value "21_" during recursive calls. so the names are printed as 21_1, 21_2 etc 

In [ ]:
# First document in the dataset accessed by array/list index 0. 
type(loans_data["data"][0])

In [ ]:
# 21 elemnt in the list accessed by index 21. 
type(loans_data["data"][0][21])

### verify 
...the output of statement print(name) in final else statement above function flatten_json(). 

```
flatten_json(loans_data["data"][0])
```

In [ ]:
flatten_json(loans_data["data"][0])

In [ ]:
import numpy as np

#creates a new dataframe that's empty
final_data=pd.DataFrame() 

# call flatten_json() method defined above for every json document in the data tag of loans_data.
# A flattened json document is supplied t json_normalize() to create a fataframe row and append it to 
# final_data dataframe.
for i in np.arange(0,len(loans_data["data"])):
    final_data=final_data.append(pd.DataFrame(json_normalize(flatten_json(loans_data["data"][i]))))

In [ ]:
# Look at the first row of data
final_data[:1]

**Reference:** [indexing pandas dataframe](http://pandas.pydata.org/pandas-docs/stable/indexing.html)

The code recursively extracts values out of the object into a flattened dictionary. json_normalize is applied to 
the output of flatten_object to produce a python dataframe. Now that we have the dataframe there are some unwanted columns that should be removed. On inspecting the view tag in dataset, unnecessary columns are identified at positions 0,1,12,14,15,16,17,18,19,20,21,22,23. So get rid of them.

In [ ]:
final_data.drop(final_data.columns[[0,1,12,14,15,16,17,18,19,20,21,22,23]],inplace=True,axis=1)
# df.columns[[0,1,12,14,15,16,17,18,19,20,21,22,23]]
final_data[:1]

Now set the column names. The **`columns`** tag in **`view`** which is in turn in **`meta`** tag has column names. Extract the names into a list called **`names`**. Find the length of columns tag to run the loop. Use len() function to find the length. 

In [ ]:
# fieldnames are present in the json object structure meta->view->columns->fieldname. So find the length of columns tag.
len(loans_data["meta"]["view"]["columns"])

    "columns" : [ {
        "id" : -1,
        "name" : "sid",
        "dataTypeName" : "meta_data",
        "fieldName" : ":sid",
        "position" : 0,
        "renderTypeName" : "meta_data",
        "format" : { },
        "flags" : [ "hidden" ]
      }, {
        "id" : -1,
        "name" : "id",
        "dataTypeName" : "meta_data",
        "fieldName" : ":id",
        "position" : 0,
        "renderTypeName" : "meta_data",
        "format" : { },
        "flags" : [ "hidden" ]
      },
      .
      .
      ,}

Above shown is the columns tag structure in the oroginal **`loans`** json object. Iterate 22 times through columns dictionary to extract the value of the key **`fieldName`**.

In [ ]:
# Since columns is an array we are using array subscript to refer to each element.

import numpy as np
names=[]
length=len(loans_data["meta"]["view"]["columns"])
for i in range(0, length):
    names.append(loans_data["meta"]["view"]["columns"][i]["fieldName"])
names

The column names with an appending **`:`** are unwanted. Get rid of those names. The forloop used below is a  consise shorthand notation of for loop. Examine the RHS. The first word, **`word`** is the output returned after evaluating the **`if`** condition.  

**Reference: ** [simplify python loops](http://blog.teamtreehouse.com/python-single-line-loops)

In [ ]:
names=[word for word in names if not ':' in word]

In [ ]:
names

The last column location is an array of length 5 with gps coordinates information. The array has latitude and longitude information. But there are also columns latitude and longitude present outside this array. These columns are filled with null values. So lets remove the column "location" from the **`names`** list and use the existing columns latitude and longitude to store gps values.

In [ ]:
# Remove last name from names list
names.pop(len(names)-1)

**Reference: **
- [Dataframe.drop()](http://chrisalbon.com/python/pandas_dropping_column_and_rows.html)

In [ ]:
type(final_data)

Use the names extracted for csv column headers. Before creating the csv file rename the columns of **`result`** dataframe. Rename the columns using dataframe.rename() function

In [ ]:
final_data.rename(index=str, columns={'8': names[0], '9': names[1], '10': names[2], '11': names[3], '12': names[4], '13': names[5],\
                              '14':names[6], '15': names[7], '16': names[8], '17': names[9], '18': names[10], \
                              '19':names[11], '20': names[12]},inplace=True)
final_data[:5]

**Reference: **
- [Dataframe.rename()](http://chrisalbon.com/python/pandas_rename_multiple_columns.html)

In [ ]:
final_data[:2]

Write the final dataframe to a csv file.

In [ ]:
import csv
import numpy as np

# index=False will not write the default index column of dataframe to csv file. 
final_data.to_csv('../../../datasets/loans.csv',sep=',',header=True,index=False)

Look at the original data [here](https://data.cityofchicago.org/Community-Economic-Development/Chicago-Microlending-Institute-CMI-Microloans/dpkg-upyz). 

## Reference :

[Excellent Python tutorial for a beginner from Google](https://developers.google.com/edu/python/introduction) 